<a href="https://colab.research.google.com/github/maxtost10/Playground-vs/blob/main/Torch_lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ein einfaches Modell mit PyTorch



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Einfaches Dataset
x = torch.rand(1000, 2)
y = ((x[:, 0] + x[:, 1]) > 1).float().unsqueeze(1)  # XOR-artig

In [3]:
dataset = TensorDataset(x, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [4]:
class SimpleNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Linear(2, 16),
        nn.ReLU(),
        nn.Linear(16, 1),
        nn.Sigmoid()
    )

  def forward(self, x):
    return self.layers(x)

In [6]:
# Training loop (PyTorch-Style)
model = SimpleNet()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)

for epoch in range(10):
  for batch_x, batch_y in dataloader:
    optimizer.zero_grad()
    output = model(batch_x)
    loss = criterion(output, batch_y)
    loss.backward()
    optimizer.step()
  print(f"Epoch {epoch}: Loss = {loss.item():.4f}")

Epoch 0: Loss = 0.7279
Epoch 1: Loss = 0.6598
Epoch 2: Loss = 0.6513
Epoch 3: Loss = 0.6589
Epoch 4: Loss = 0.6517
Epoch 5: Loss = 0.6502
Epoch 6: Loss = 0.7145
Epoch 7: Loss = 0.7235
Epoch 8: Loss = 0.6168
Epoch 9: Loss = 0.6830


# Dasselbe mit PyTorch Lightning

In [8]:
!pip install pytorch_lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 22.0 MB/s eta 0:00:00


In [10]:
import pytorch_lightning as pl

class LitSimpleNet(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Linear(2, 16),
        nn.ReLU(),
        nn.Linear(16, 1),
        nn.Sigmoid()
    )
    self.loss_fn = nn.BCELoss()

  def forward(self, x):
    return self.layers(x)

  def training_step(self, batch, batch_idx):
    x, y = batch
    y_hat = self(x)
    loss = self.loss_fn(y_hat, y)
    self.log('train_loss', loss)
    return loss

  def configure_optimizers(self):
    return optim.Adam(self.parameters(), lr = 0.01)

# Lightning trainer
from pytorch_lightning import Trainer

model = LitSimpleNet()
trainer = Trainer(max_epochs=100, log_every_n_steps=10)
trainer.fit(model, dataloader)

INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: True, using: 8 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | layers  | Sequential | 65     | train
1 | loss_fn | BCELoss    | 0      | train
-----------------------------------------------
65        Trainable params
0         Non-trainable params
65        Total params
0.000     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]